# Exploratory Data Analysis and Visualization

## Objective

The objective in this notebook is as follows: 
1. Load the notebook from the web
2. Clean and wrangle the data into a tidy format
3. Propose a visualization that is relevant to addressing the question and to explore the data.
    * propose a high quality plot or set of plots of the same kind
    * explain why it is relevant in addressing the question or exploring the data

In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# >>> Define paths to repository and data folder <<< #
repo_path  <- "https://raw.githubusercontent.com/fuminaba/STAT-301"
data_parent_path <- "/fuminaba-project/Data"

# >>> Load Training and Testing Data <<< #
data.train <- paste0(repo_path, data_parent_path, "/adult.data")  %>% 
    read_csv(col_names = F)
data.test  <- paste0(repo_path, data_parent_path, '/adult.test') %>% 
    read_csv(col_names = F, skip = 1)

# >>> Define feature names and rename columns <<< #
feature_names <- c('age', 'workclass', 'fnlwgt', 'education',
                   'education-num', 'marital-status', 'occupation',
                   'relationship', 'race', 'sex', 'capital-gain', 
                   'capital-loss', 'hours-per-week', 'native-country',
                   'income')

names(data.train) <- feature_names
names(data.test) <- feature_names

# >>> For EDA, we will combine train and test data <<< #
data.all <- rbind(data.train, data.test)

Rows: 32561 Columns: 15
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): X2, X4, X6, X7, X8, X9, X10, X14, X15
dbl (6): X1, X3, X5, X11, X12, X13

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 16281 Columns: 15
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): X2, X4, X6, X7, X8, X9, X10, X14, X15
dbl (6): X1, X3, X5, X11, X12, X13

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
invisible(capture.output(DataExplorer::create_report(data = data.all)))



processing file: report.rmd

output file: D:/Fumi/STAT-301/2-EDA-and-Visualization/report.knit.md



Output created: D:/Fumi/STAT-301/2-EDA-and-Visualization/report.html

